In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!ls "./drive/My Drive/Train_data"

cherry	strawberry  tomato


In [3]:
!pip install imutils

In [0]:
import numpy as np
import random
import cv2
from imutils import paths
import os
from sklearn.preprocessing import LabelBinarizer
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import (densenet,
                                           inception_resnet_v2,
                                           inception_v3,
                                           nasnet,
                                           resnet50,
                                           vgg16,
                                           vgg19,
                                           xception)
import tensorflow.keras as K

# Set random seeds to ensure the reproducible results
SEED = 309
np.random.seed(SEED)
random.seed(SEED)


In [0]:
t_size = [2000, 100]
WIDTH, HEIGHT = 300,300

In [0]:
ps_up = sorted(list(paths.list_images("./drive/My Drive/Train_data")))
ps_up = [ps.replace("\\","") for ps in ps_up]

# build a range as pointers for image files
temp = range(0, len(ps_up))

# randomly sample for our training set
# needs to be big enough to be have capacity for a validation set
train_indices = np.random.choice(temp, size = t_size[0])
train = [ps_up[i] for i in train_indices]

In [0]:
lbls, imgs = [],[]
for imagePath in train:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)

In [0]:
X_train = np.array(imgs, dtype="float") / 255.0
y_train = np.array(lbls)

# Binarize the labels
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)

In [0]:
# exclude those training images from our test set
temp = [i for i in temp if i not in train]
# subset for testing
test_indices = np.random.choice(temp, size = t_size[1])
test = [ps_up[i] for i in test_indices]

lbls, imgs = [],[]

for imagePath in test:
  image = cv2.imread(imagePath)
  # load the image, pre-process it, and store it in the data list\
  image = cv2.resize(image, (WIDTH, HEIGHT))
  image = img_to_array(image)
  imgs.append(image)
  label = imagePath.split(os.path.sep)[-2]
  lbls.append(label)



In [0]:
X_test = np.array(imgs, dtype="float") / 255.0
y_test = np.array(lbls)

# Binarize the labels
# lb = LabelBinarizer()
y_test = lb.fit_transform(y_test)

In [8]:
print(len(X_train),
      len(y_train),
      len(X_test),
      len(y_test))

2000 2000 100 100


This is the switch for changing the model. The colab engine is memory restricted so can't take more than one in memory at any time. 

The instance needs to be terminated between models (to free up resources).

In [0]:
model_name = 'xcept'
# # models
# #     'dense'
# instance = densenet.DenseNet169(weights='imagenet', input_shape=(300, 300 ,3), include_top=False)
# #     'incep_v3'
# instance = inception_v3.InceptionV3(weights='imagenet', input_shape=(300, 300 ,3), include_top=False)
# #     'nas'
# instance = nasnet.NASNetMobile(weights='imagenet', input_shape=(300, 300 ,3), include_top=False)
# #     'res' 
# instance = resnet50.ResNet50(weights='imagenet', input_shape=(300, 300 ,3), include_top=False)
# #     'xcept'
instance = xception.Xception(weights='imagenet', input_shape=(300, 300 ,3), include_top=False)

In [10]:
stack = K.models.Sequential()
stack.add(instance)
stack.add(K.layers.Flatten())
stack.add(K.layers.Dense(1024, activation='relu'))
stack.add(K.layers.Dropout(0.5))
stack.add(K.layers.Dense(3, activation='sigmoid'))

stack.compile(
    optimizer=K.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

stack.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=4
)

score = stack.evaluate(X_test, y_test)

Train on 1600 samples, validate on 400 samples
Epoch 1/5
1600/1600 [==============================] - 273s 171ms/step - loss: 1.3032 - acc: 0.3981 - val_loss: 1.1474 - val_acc: 0.3875
Epoch 2/5
1600/1600 [==============================] - 257s 161ms/step - loss: 1.1976 - acc: 0.4250 - val_loss: 1.0955 - val_acc: 0.3525
Epoch 3/5
1600/1600 [==============================] - 257s 160ms/step - loss: 1.2945 - acc: 0.3781 - val_loss: 1.3127 - val_acc: 0.3000
Epoch 4/5
1600/1600 [==============================] - 257s 160ms/step - loss: 1.6970 - acc: 0.3581 - val_loss: 1.1344 - val_acc: 0.3850
Epoch 5/5
100/100 [==============================] - 3s 32ms/step


In [11]:
from pandas import DataFrame
score = DataFrame(score)
score = score.transpose()
score.columns = stack.metrics_names
score['Model'] = model_name
score.to_csv("./drive/My Drive/scores_comparison.csv", mode='a+')
score

,loss,acc,Model
0,1.085808,0.38,xcept
